# Creating Spark Session with Delta Extensions

In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("learningDeltaTables") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/moteroperdido/Desktop/projects/learning/learning_delta_tables/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/moteroperdido/.ivy2/cache
The jars for the packages stored in: /Users/moteroperdido/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13d14676-efb8-4e52-8c6e-cdd43ebb968a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 72ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     | 

In [2]:
spark

# Create example table

In [4]:
data = spark.range(0, 5)
data.write.format("delta").save("./data/delta-table")

23/09/06 13:38:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
data.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



# Read the table

In [7]:
df = spark.read.format("delta").load("./data/delta-table")
df.show()

+---+
| id|
+---+
|  3|
|  2|
|  4|
|  0|
|  1|
+---+



# Overwrite table data

In [8]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("./data/delta-table")

In [10]:
spark.read.format("delta").load("./data/delta-table").show()

+---+
| id|
+---+
|  7|
|  5|
|  9|
|  6|
|  8|
+---+



# Time travel

In [14]:
df = spark.read.format("delta").option("versionAsOf", 0).load("./data/delta-table")
df.show()

+---+
| id|
+---+
|  3|
|  2|
|  4|
|  0|
|  1|
+---+



In [19]:
data = spark.range(5, 10).coalesce(1)
data.write.format("delta").mode("overwrite").save("./data/delta-table")